In [1]:
!pip freeze | grep scikit-learn


scikit-learn @ file:///private/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_2184e3zm58/croot/scikit-learn_1694788538013/work


In [2]:
!python -V

Python 3.9.18


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/christopherlewis/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/christopherlewis/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:

import pyarrow
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject


In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
print(f'Q1: The standard deviation of the predicted trip duration is {y_pred.std():.2f} minutes.')

Q1: The standard deviation of the predicted trip duration is 6.25 minutes.


In [8]:
df['ride_id'] = df.tpep_pickup_datetime.dt.strftime('%Y/%m') + '_' + df.index.astype(str)

In [9]:
ride_ids = df.ride_id.to_list()
ride_dict = dict(zip(ride_ids, y_pred))
df_results = pd.DataFrame.from_dict(ride_dict, orient='index', columns=["DurationPrediction"])
df_results.reset_index(inplace=True)
df_results.rename(columns={'index': 'Ride_Id', 'DurationPrediction': 'DurationPrediction'}, inplace=True)

df_results.head()


,Ride_Id,DurationPrediction
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [10]:
df_results.to_parquet(
    'duration_dict.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [11]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1850 bytes to starter.py


# The terminal shows that the resulting file is 65M

## Q.3. The command is:  jupyter nbconvert --to script starter.ipynb


## Q.4.  The first hash of the scikit-learn dependency is:   "sha256:0e8102d5036e28d08ab47166b48c8d5e5810704daecf3a476a4282d562be9a28"

## Q.5. The mean predicted trip duration is 14.292 minutes.

## Q.6. The mean predicted trip duration is 0.192 minutes.